# Классификация изображений

### Основная идея этого решения: взять предобученую на ImageNet сеть и дообучить под нашу задачу. 

В этом ноутбуке описано применение finetuning и переноса обучения.

Установим все необходимые библиотеки:

In [ ]:
#вот тут неплохо описано зачем эта команда https://andreyv.ru/nvidia-smi-poleznye-komandy.html
!nvidia-smi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.efficientnet import EfficientNetB6
from tensorflow.keras.applications.efficientnet import EfficientNetB4
from tensorflow.keras.layers import *

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import RemoteMonitor
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import ProgbarLogger


from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL #Python Imaging Library
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [ ]:
!pip freeze > requirements.txt

# Основные настройки

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 5 # эпох на обучение
BATCH_SIZE           = 16 #32 #64 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 128 # какого размера подаем изображения в сеть
#IMG_SIZE =           (384, 512)
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
#input_shape          = (IMG_SIZE[0], IMG_SIZE[1], IMG_CHANNELS)

DATA_PATH = '../input/'
PATH = "../working/car/" # рабочая директория


Были опробованы следующие варианты параметров:
*     EPOCHS = 5 и 10, чем больше эпох, тем выше скор, но падает скорость обучения
*     BATCH_SIZE = 16 и 32 и 64, принудительно приходилось уменьшать, если сеть большая, иначе не влезет в память на GPU
* IMG_SIZE = (384, 512) - самый лучший результат, пробовала варианты 128/224/384/512, когда длина и ширина изображения совпадают, для теста использовала 128 (так быстрее)


    


In [ ]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(PATH,exist_ok=False)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное - это хорошо

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    #To load an image from a file, use the open() function in the Image module:
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
#Once you have an instance of the Image class, you can use the methods defined by this class to process 
#and manipulate the image. For example, let’s display the image we just loaded:
imgplot = plt.imshow(image)
plt.show()
image.size

# Подготовка данных

### Аугментация данных

**исходный вариант** - лучший результат для изображения 384*512

In [ ]:
# Официальная документация: https://keras.io/preprocessing/image/
#ImageDataGenerator Generate batches of tensor image data with real-time data augmentation.

#The data will be looped over (in batches).
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=VAL_SPLIT, # set validation split
    horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1. / 255) # вариант для сетей Xception и InceptionResNetV2
#test_datagen = ImageDataGenerator() # без нормализации для сети EfficientNet
#Рекомендация Подключите более продвинутые библиотеки аугментации изображений (например: albumentations или imgaug, для них есть специальные "обертки" под Keras, например: https://github.com/mjkvaak/ImageDataAugmentor)

**вариант 2 с подобранными значениями** - лучший результат для изображения 512*512

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 5, #Int. Degree range for random rotations. было 5, увеличим до 10,скор хуже, вернула 5
    width_shift_range=0.2, # доля от общей ширины было 0.1, увеличим до 0.2 скор лучше
    height_shift_range=0.2, # доля от общей высоты было 0.1, увеличим д0 0.2 скор лучше
    validation_split=VAL_SPLIT, # set validation split
    horizontal_flip=True) #Boolean. Randomly flip inputs horizontally.было False,скор лучше
#vertical_flip=False, # Boolean. Randomly flip inputs vertically.по вертикали смысла нет, зачем машины вверх ногами
#fill_mode = "nearest" пробовала все варианты, по умолчанию лучше, чем constant","reflect" и "wrap
#

test_datagen = ImageDataGenerator(rescale=1. / 255)

Изображение, считанное с диска, имеет формат uint8 (то есть unsigned integer 8-bit). Пиксели принимают целочисленные значения от 0 до 255. Операция нормализации (см. строку кода выше) переводит пиксели в формат float (число с плавающей запятой), и пиксели теперь принимают значения от 0 до 1. Это и называется нормализацией. Так вот, для Xception и InceptionResNetV2 нормализация нужна, а для EfficientNet не нужна.

Однако некоторые из этих архитектур имеют особенности. Например используя EfficientNetB0 ... EfficientNetB7 нужно пропускать шаг нормализации изображения, то есть на вход нейросети подавать изображения в формате uint8 от 0 до 255

ImageDataGenerator хранит информацию о том, какие преобразования выполнять с изображениями, но не хранит информацию о том, откуда их брать.
При вызове метода flow_from_directory указывается, откуда брать изображения, каков размер батча и другие параметры. В итоге train_generator хранит информацию о том, откуда брать изображения и какие преобразования выполнять. Это не массив и даже не итератор, это просто набор информации. Каким же образом получать из него изображения?
Объект train_generator можно перебрать например циклом for:
for element in train_generator:
    pass
Если рассматривать устройство языка Python, то train_generator является объектом типа iterable, то есть имеет метод __iter__() . Вызвав этот метод, мы получим итератор. Итератор хранит позицию текущего элемента. Рекомендую почитать в интернете о том, как работают в питоне итераторы.
Иными словами, создавая train_generator мы указываем, что некоторые данные хранятся на диске, и указываем, как с ними работать. А именно мы говорим, что каждый раз получая из train_generator изображения мы применяем к ним аугментации. Например:
def get_all_data():
    return list(train_generator)
Таким образом мы загрузим все данные (изображения и метки) в массив. При каждом вызове метода get_all_data() мы будем получать массив изображений одной и той же длины, но с разными аугментациями. 
Выполнив код:
element = train_generator.__iter__().__next__()
мы получим первый элемент из генератора. Это кортеж (tuple) из двух элементов: батча изображений и батча меток. Поэтому можно даже сделать так:
images_batch, labels_batch = train_generator.__iter__().__next__()

Если суммировать все сказанное: при создании train_generator не создаются копии изображений. Они будут создаваться тогда, когда метод .fit() будет перебирать train_generator. На каждой эпохе он будет перебирать его заново и получать изображения с другими аугментациями.

### Аугментация данных с помощью albumentations

In [ ]:
#вариант реализации взят отсюда https://github.com/mjkvaak/ImageDataAugmentor/blob/master/examples/classification-with-flow_from_directory.ipynb
AUGMENTATIONS = albumentations.Compose([
    # flips
    albumentations.HorizontalFlip(p=0.25),
    #albumentations.VerticalFlip(p=0.25),закомментировала вертикальный поворот, изображение вверх ногами имеет сомнительную ценность
    # color augmentations
    albumentations.OneOf([
        albumentations.HueSaturationValue(p=1.),
        albumentations.RandomBrightnessContrast(p=1.),
        albumentations.RGBShift(p=1.)
    ], p=0.25),
    # image quality
    albumentations.OneOf([
        albumentations.GaussNoise(p=1.),
        albumentations.MultiplicativeNoise(p=1.),
        albumentations.JpegCompression(p=1.),
        albumentations.Downscale(scale_min=0.5,scale_max=0.99, p=1),
    ], p=0.5),
    # other
    albumentations.OneOf([
        albumentations.ToGray(p=1.0),
        albumentations.RandomResizedCrop(height=IMG_SIZE, 
                                         width=IMG_SIZE),
        
    ], p=0.25),
])

train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        validation_split=VAL_SPLIT,
        )
        
test_datagen = ImageDataAugmentor(rescale=1./255)

In [ ]:
#вариант реализации взят отсюда https://github.com/mjkvaak/ImageDataAugmentor
AUGMENTATIONS = albumentations.Compose([
    albumentations.Transpose(p=0.5),
    albumentations.Flip(p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=1),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
])

train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        validation_split=VAL_SPLIT,
        )
        
test_datagen = ImageDataAugmentor(rescale=1./255)


### Генерация данных

In [ ]:
# Завернем наши данные в генератор:
#flow_from_directory - Takes the path to a directory & generates batches of augmented data.
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    #target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    #target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    #target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    batch_size=BATCH_SIZE,)

# Обратите внимание, что для сабмита мы используем другой источник test_datagen.flow_from_dataframe.
#Потому что нам нужно дать предсказание с привязкой к конкретному Id картинки из dataframe.

In [ ]:
# посмотрим на результаты аугментации
train_generator.show_data()

# Построение модели

### Загружаем предобученные сети:

In [ ]:
base_model = Xception(
    weights='imagenet', # Подгружаем веса imagenet
    include_top=False, # Выходной слой (голову) будем менять т.к. у нас други классы
    input_shape = input_shape)

In [ ]:
base_model = EfficientNetB4(
    include_top=False,  # Выходной слой (голову) будем менять т.к. у нас други классы
    weights="imagenet", # Подгружаем веса imagenet
    input_shape = input_shape
)

при прочих равных условиях (кол-во эпох, размер изображения, вариант аугментации и т.д.) результат лучше, чем Xception на изображениях размером 128*128, на изображениях большего размера возникает ошибка ResourceExhaustedError

In [ ]:
base_model = EfficientNetB6(
    weights='imagenet', # Подгружаем веса imagenet
    include_top=False,  # Выходной слой (голову) будем менять т.к. у нас други классы
    input_shape=input_shape
)

при прочих равных условиях (кол-во эпох, размер изображения, вариант аугментации и т.д.) результат лучше, чем Xception, но хуже, чем у EfficientNetB4

In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape = input_shape)

при прочих равных условиях (кол-во эпох, размер изображения, вариант аугментации и т.д.) результат с базовой моделью InceptionResNetV2 хуже, даже чем Xception

In [ ]:
base_model.summary()

In [ ]:
# Устанавливаем новую "голову" (head)

x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#вот тут надо менять в зависимости от размера изображения 
x = Dense(128, activation='relu')(x)
#x = Dense(IMG_SIZE[0],activation='relu')(x)
#добавила Batch Normalization
x =BatchNormalization()(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 10 classes
predictions = Dense(CLASS_NUM, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

Добавление BatchNormalization немного улушило скор

это фукнциональный вариант сборки модели, попробуем еще последовательный вариант. Функциональные модели гибче, можно не только связывать слой с предыдущим и последующим. Вообще рекомендую использовать функциональный стиль только тогда, когда по-другому невозможно, потому что функциональный стиль хуже читаем и повышает шанс ошибиться.

вариант последовательной модели (sequential) - с таким вариантом получен самый высокий скор 

In [ ]:
model = tf.keras.Sequential()

model.add(base_model)
model.add(GlobalAveragePooling2D(),)
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


#операция с Sequential-моделью. 
predictions = model.add(Dense(CLASS_NUM, activation='softmax'))

#Если у нас Sequantial-модель, то строка ниже не нужна
#model = Model(inputs=base_model.input, outputs=predictions)

#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

этот вариант последовательной модели взят из одного из заданий, для проверки

In [ ]:
    #из DL3
model = tf.keras.Sequential()
model.add(base_model)
#Conv2D - это сверточный слой
model.add(Conv2D(16, kernel_size=3, strides=1, padding='same', input_shape=input_shape))
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same'))
model.add(MaxPool2D())
#для регуляризации сети слой  Dropout
model.add(Dropout(0.25))
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same'))
model.add(Conv2D(64, kernel_size=3, strides=1, padding='same'))
model.add(MaxPool2D())
model.add(Dropout(0.25))
    #уплощение - вытягивание в вектор
model.add(Flatten())
    # чтобы получить больше параметров, добавим ещё один плотный слой перед крайним
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
    #полносвязный слой - плотный слой у которого 10 выходов (у нас 10 классов) и функция активации softmax
predictions = model.add(Dense(CLASS_NUM, activation='softmax'))
    
#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

На последнем слое нельзя делать активацию relu, потому что все выходные значение меньше нуля обрезаются, то есть теряется информация на выходе. В промежуточных слоях relu используется для создания нелинейности, но в выходном слое нет смысла его использовать. Можно не указывать activation, либо указать activation='linear', что то же самое или softmax

функция потерь "categorical_crossentropy", что то же самое, что tf.keras.losses.CategoricalCrossentropy(), но эта функция потерь подразумевает, что на выходном слое у вас активация softmax. Если не использовать softmax, то нужно указать параметр: tf.keras.losses.CategoricalCrossentropy(from_logits = True)

In [ ]:
model.summary()

In [ ]:
# сколько слоев
print(len(model.layers))

In [ ]:
# Количество параметров обучения
len(model.trainable_variables)

In [ ]:
# Статус слоев - будем обучать или нет
for layer in model.layers:
    print(layer, layer.trainable)

## Обучение модели

**Step 1 - обучение "головы"**

In [ ]:
LR=0.001
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')

#Enable visualizations for TensorBoard.
tensboard= TensorBoard(
    log_dir="logs",
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    update_freq="epoch",
    profile_batch=2,
    embeddings_freq=0,
    embeddings_metadata=None)

#Stop training when a monitored metric has stopped improving. - вот это оч круто! останавливает обучение, когда
#метрика перестает улучшаться, то есть сильно экономит время!
earlystop = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)
#Learning rate scheduler
#scheduler = LearningRateScheduler(schedule, verbose=0)
#https://keras.io/api/callbacks/learning_rate_scheduler/ надо функцию писать для шедулера

#Reduce learning rate when a metric has stopped improving.
reduceplateau = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0)

#Callback used to stream events to a server.
#monitor = RemoteMonitor(
    #root="http://localhost:9000",
    #path="/publish/epoch/end/",
    #field="data",
    #headers=None,
    #send_as_json=False)
    
#Callback for creating simple, custom callbacks on-the-fly.
#lambda = LambdaCallback(
    #on_epoch_begin=None,
    #on_epoch_end=None,
    #on_batch_begin=None,
    #on_batch_end=None,
    #on_train_begin=None,
    #on_train_end=None)

#Callback that prints metrics to stdout.
#logger = ProgbarLogger(count_mode="samples", stateful_metrics=None)


callbacks_list = [checkpoint, tensboard, earlystop, reduceplateau]


# Рекомендация 2. Используйте разные техники управления Learning Rate
# https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6 (eng)
# http://teleported.in/posts/cyclic-learning-rate/ (eng)

Обучаем:

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
#Saves the model to Tensorflow SavedModel or a single HDF5 file.
model.save('../working/model_last.hdf5')
#Loads all layer weights, either from a TensorFlow or an HDF5 weight file.
model.load_weights('best_model.hdf5')

In [ ]:
#evaluate_generator - это один из методов, как fit/predict
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

  Посмотрим графики обучения:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

**Step 2 - FineTuning - обучение половины весов**

In [ ]:
# Посмотрим на количество слоев в базовой модели
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
#Разморозим базовую модель
base_model.trainable = True

# Установим количество слоев, которые будем переобучать
fine_tune_at = len(base_model.layers)//2

# Заморозим первую половину слоев
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
# Количество параметров
len(base_model.trainable_variables)

In [ ]:
# Статус слоев - будем обучать или нет
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
LR=0.0001
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
callbacks_list = [checkpoint, tensboard, earlystop, reduceplateau]

In [ ]:
# Обучаем
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples//test_generator.batch_size,
    epochs = EPOCHS,
    callbacks = callbacks_list
)

In [ ]:
model.save('../working/model_step2.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
#evaluate_generator - это один из методов, как fit/predict
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

**Step 3 - FineTuning - разморозка всей сети и дообучение**

In [ ]:
#Разморозим базовую модель
base_model.trainable = True

In [ ]:
LR=0.00001
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
callbacks_list = [checkpoint, tensboard, earlystop, reduceplateau]

In [ ]:
# Обучаем
history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
model.save('../working/model_step3.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
#evaluate_generator - это один из методов, как fit/predict
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

**Step 4 - увеличение размера изображения**

In [ ]:
EPOCHS               = 10
BATCH_SIZE           = 4 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-4

IMG_SIZE             = 512
IMG_CHANNELS         = 3
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

In [ ]:
#лучший вариант параметров для изображения 512*512
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 5, #Int. Degree range for random rotations. было 5, увеличим до 10,скор хуже, вернула 5
    width_shift_range=0.2, # доля от общей ширины было 0.1, увеличим до 0.2 скор лучше
    height_shift_range=0.2, # доля от общей высоты было 0.1, увеличим д0 0.2 скор лучше
    validation_split=VAL_SPLIT, # set validation split
    horizontal_flip=True) #Boolean. Randomly flip inputs horizontally.было False,скор лучше
#vertical_flip=False, # Boolean. Randomly flip inputs vertically.по вертикали смысла нет, зачем машины вверх ногами
#fill_mode = "nearest" пробовала все варианты, по умолчанию лучше, чем constant","reflect" и "wrap
#

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# Завернем наши данные в генератор:
#flow_from_directory - Takes the path to a directory & generates batches of augmented data.
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    #target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    #target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    #target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    batch_size=BATCH_SIZE,)

In [ ]:
#Заново создаем сеть с новым размером входных данных
base_model = Xception(
    weights='imagenet', # Подгружаем веса imagenet
    include_top=False, # Выходной слой (голову) будем менять т.к. у нас други классы
    input_shape = input_shape)

In [ ]:
model = tf.keras.Sequential()

model.add(base_model)
model.add(GlobalAveragePooling2D(),)
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


#операция с Sequential-моделью. 
predictions = model.add(Dense(CLASS_NUM, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
callbacks_list = [checkpoint, tensboard, earlystop, reduceplateau]

In [ ]:
# Обучаем
history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
        )

In [ ]:
model.save('../working/model_step4.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Предсказание на тестовых данных

In [ ]:
#это результат "Завернем наши данные в генератор"
test_sub_generator.samples

In [ ]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

# Рекомендация: попробуйте добавить Test Time Augmentation (TTA)
# https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

In [ ]:
submission.head()

In [ ]:
# Clean PATH
import shutil
shutil.rmtree(PATH)

shutil.rmtree(path, ignore_errors=False, onerror=None) - Удаляет текущую директорию и все поддиректории; path должен указывать на директорию, а не на символическую ссылку.